# Embeddings Contextuales con FinBERT

En este notebook se generan embeddings contextuales para las noticias utilizando el modelo FinBERT, especializado en texto financiero. Se utiliza el token CLS como representación del texto completo.

Importar todas las librerías necesarias

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import os
import numpy as np

/home/adiez/anaconda3/envs/nlp_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Definir directorios de donde queremos cargar los datos resumidos y donde queremos guardar los embeddings

In [2]:
# Usa el directorio actual del notebook como raíz del proyecto
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Define las rutas a los directorios de datos
SUMMARIES_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'summarized')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'data', 'news', 'embeddings')

# Crea el directorio de salida si no existe
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Lista todos los archivos .parquet en el directorio SUMMARIES_DIR
parquet_files = [f for f in os.listdir(SUMMARIES_DIR) if f.endswith('.parquet')]

# Muestra el número total de archivos encontrados
print(f"Se han encontrado {len(parquet_files)} archivos parquet para procesar.\n")

# Lista los nombres de los archivos encontrados
for file in parquet_files:
    print(file)

Se han encontrado 7 archivos parquet para procesar.

amazon_news.parquet
apple_news.parquet
google_news.parquet
meta_news.parquet
microsoft_news.parquet
nvidia_news.parquet
tesla_news.parquet


Cargar modelo FinBERT y tokenizador

In [3]:
print("Cargando modelo FinBERT...")
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert")
model.eval()
print("Modelo FinBERT cargado.")

Cargando modelo FinBERT...
Modelo FinBERT cargado.
Modelo FinBERT cargado.


Configurar dispositivo para procesamiento (GPU o CPU)

In [4]:
# Mover modelo a GPU si está disponible
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


Definir función para generar embeddings contextuales usando el token CLS de FinBERT

In [5]:
def get_text_embedding(text):
    """
    Genera embedding contextual para un texto usando FinBERT.
    Retorna el embedding del token [CLS] como representación del texto.
    """
    # Tokenizar y preparar inputs
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Obtener embeddings
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Usar embedding del token [CLS] (primer token) como representación del texto
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    
    return cls_embedding.cpu().numpy()

Procesamiento de archivos: para cada archivo se realiza la carga de datos, concatenación de título y resumen, generación de embeddings contextuales y guardado de resultados

In [6]:
for file_idx, file_name in enumerate(parquet_files, 1):
    print(f"[{file_idx}/{len(parquet_files)}] Procesando {file_name}...")
    
    # Cargar dataset
    df = pd.read_parquet(os.path.join(SUMMARIES_DIR, file_name), engine="pyarrow")
    
    # Concatenar título y resumen
    df["text"] = df["title"] + "\n\n" + df["body_summary"]
    df = df.drop(columns=["clean_body", "body_summary", "teaser", "title", "author"])
    
    # Eliminar filas con texto vacío
    original_count = len(df)
    df = df.dropna(subset=['text'])
    final_count = len(df)
    
    if original_count != final_count:
        print(f"  Se han eliminado {original_count - final_count} filas con texto vacío")
    
    print(f"  Generando embeddings para {final_count} textos...")
    
    # Generar embeddings con seguimiento de progreso
    embeddings = []
    for idx, text in enumerate(df['text'], 1):
        if idx % 100 == 0 or idx == final_count:
            print(f"    Progreso: {idx}/{final_count} ({100*idx/final_count:.1f}%)", end='\r')
        embeddings.append(get_text_embedding(text))
    
    df['embedding'] = embeddings
    print()  # Nueva línea después del progreso
    
    # Guardar en directorio de salida
    output_name = file_name.replace('_news.parquet', '_embeddings_contextual.parquet')
    output_path = os.path.join(OUTPUT_DIR, output_name)
    df.to_parquet(output_path, engine="pyarrow", index=False)
    
    embedding_dim = len(df['embedding'].iloc[0])
    print(f"  Guardado {output_name} con {len(df)} registros y dimensión de embedding {embedding_dim}\n")

print("Todos los archivos han sido procesados exitosamente!")
print("Archivos guardados en:", OUTPUT_DIR)

[1/7] Procesando amazon_news.parquet...
  Generando embeddings para 12391 textos...
    Progreso: 12391/12391 (100.0%)
    Progreso: 12391/12391 (100.0%)
  Guardado amazon_embeddings_contextual.parquet con 12391 registros y dimensión de embedding 768

[2/7] Procesando apple_news.parquet...
  Guardado amazon_embeddings_contextual.parquet con 12391 registros y dimensión de embedding 768

[2/7] Procesando apple_news.parquet...
  Generando embeddings para 24208 textos...
  Generando embeddings para 24208 textos...
    Progreso: 24208/24208 (100.0%)
    Progreso: 24208/24208 (100.0%)
  Guardado apple_embeddings_contextual.parquet con 24208 registros y dimensión de embedding 768

[3/7] Procesando google_news.parquet...
  Generando embeddings para 13341 textos...
  Guardado apple_embeddings_contextual.parquet con 24208 registros y dimensión de embedding 768

[3/7] Procesando google_news.parquet...
  Generando embeddings para 13341 textos...
    Progreso: 13341/13341 (100.0%)
    Progreso: 133